In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='databricks-scope-app'),
 SecretScope(name='databricks-streaming')]

In [0]:
spark.conf.set(
    "fs.azure.account.key.adlsg2dataengineer.dfs.core.windows.net",
    dbutils.secrets.get(scope="databricks-streaming", key="adlsg2-key")
)

## Streaming data frame

In [0]:
df=spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format","parquet")\
        .option("cloudFiles.schemaLocation","abfss://destination@adlsg2dataengineer.dfs.core.windows.net/schema")\
            .load("abfss://source@adlsg2dataengineer.dfs.core.windows.net/")

In [0]:
df.writeStream.format("delta")\
    .option("checkpointLocation","abfss://destination@adlsg2dataengineer.dfs.core.windows.net/schema")\
    .trigger(processingTime="10 seconds")\
        .start("abfss://destination@adlsg2dataengineer.dfs.core.windows.net/data")
